# Importing packages

In [ ]:
import torch

from tokenizers import Tokenizer

from nmt.utils import (
    model as model_util,
    config as config_util,
)
from nmt.translator import Translator

# Loading config file

In [2]:
config_path = './config/inference.yaml'
config = config_util.get_config(config_path)
config

{'model_path': './_test_translate/transformer-nmt-base/transformer_nmt_base_09.pt',
 'src_tokenizer': './_test_translate/transformer-nmt-base/tokenizer_source.json',
 'target_tokenizer': './_test_translate/transformer-nmt-base/tokenizer_target.json',
 'd_model': 512,
 'num_layers': 6,
 'num_heads': 8,
 'd_ffn': 2048,
 'seq_length': 120,
 'beam_size': 4,
 'beam_return_topk': 3,
 'preprocess': {'lowercase': True, 'contractions': True},
 'postprocess': {'remove_underscores': True}}

# Building `Translator` 

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device = torch.device(device)

print('Loading tokenizers')
src_tokenizer = Tokenizer.from_file(config['src_tokenizer'])
target_tokenizer = Tokenizer.from_file(config['target_tokenizer'])

model = model_util.make_model(src_tokenizer, target_tokenizer, config)
model.to(device)

states = torch.load(config['model_path'], map_location=device)

model.load_state_dict(states['model_state_dict'])

translator = Translator(model, src_tokenizer, target_tokenizer, config)

Loading tokenizers
Model has 80904084 learnable parameters


In [4]:
translator("hello, world!", beam_size=config['beam_size'], beam_return_topk=config['beam_return_topk'], max_seq_length=config['seq_length'])

['xin chào, thế giới!', 'xin chào thế giới!', 'chào thế giới!']

In [5]:
translator("Does he know about this?", beam_size=config['beam_size'], beam_return_topk=config['beam_return_topk'], max_seq_length=config['seq_length'])

['anh ta có biết về điều này không?',
 'anh ta có biết không?',
 'anh ấy có biết về điều này không?']

In [6]:
translator("i don't know how to do that.", beam_size=config['beam_size'], beam_return_topk=config['beam_return_topk'], max_seq_length=config['seq_length'])

['tôi không biết làm thế nào.',
 'tôi không biết làm thế nào để làm điều đó.',
 'tôi không biết làm sao để làm điều đó.']

In [7]:
translator("i don't know how to do these.", beam_size=config['beam_size'], beam_return_topk=config['beam_return_topk'], max_seq_length=config['seq_length'])

['tôi không biết làm thế nào.',
 'tôi không biết làm thế nào để làm những điều này.',
 'tôi không biết làm thế nào để làm được điều này.']

In [8]:
translator("it's broken, great!", beam_size=config['beam_size'], beam_return_topk=config['beam_return_topk'], max_seq_length=config['seq_length'])

['nó bị gãy, tuyệt!', 'nó bị gãy, tuyệt vời!', 'nó bị hỏng, tuyệt!']

In [9]:
translator("You would be surprised how kids are affected by this", beam_size=config['beam_size'], beam_return_topk=config['beam_return_topk'], max_seq_length=config['seq_length'])

['bạn sẽ ngạc nhiên khi trẻ em bị ảnh hưởng bởi điều này.',
 'bạn sẽ ngạc nhiên làm sao những đứa trẻ bị ảnh hưởng bởi điều này.',
 'bạn sẽ ngạc nhiên khi những đứa trẻ bị ảnh hưởng bởi điều này.']

In [11]:
translator("You would be surprised how kids are affected by this", beam_size=config['beam_size'], beam_return_topk=config['beam_return_topk'], max_seq_length=2)


['bạn sẽ', 'các bạn', 'bạn có thể']